In [1]:
#import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, evaluate
sns.set_style("darkgrid")
from collections import deque

In [2]:
#####Data Import 
#This step is to icorporate movie ID into the original data 
#Reference: https://www.kaggle.com/morrisb/how-to-recommend-anything-deep-recommender
#laod the user data
# Load single data-file
dfuser = pd.read_csv('combined_data_1.txt', header=None, names=['User', 'Rating', 'Date'], usecols=[0, 1, 2])
# Find empty rows to slice dataframe for each movie
tmpmovies = dfuser[dfuser['Rating'].isna()]['User'].reset_index()
movieindice = [[index, int(movie[:-1])] for index, movie in tmpmovies.values]
# Shift the movie_indices by one to get start and endpoints of all movies
shifted = deque(movieindice)
shifted.rotate(-1)
# Gather all dataframes
user_data = []
# Iterate over all movies
for [df_id_1, movieid], [df_id_2, nextmovieid] in zip(movieindice, shifted): 
    # Check if it is the last movie in the file
    if df_id_1<df_id_2:
        tmp_df = dfuser.loc[df_id_1+1:df_id_2-1].copy()
    else:
        tmp_df = dfuser.loc[df_id_1+1:].copy()  
    # Create movie_id column
    tmp_df['Movie'] = movieid
    # Append dataframe to list
    user_data.append(tmp_df)
# Combine all dataframes
newdf = pd.concat(user_data)
del user_data, dfuser, tmpmovies, tmp_df, shifted, movieindice, df_id_1, movieid, df_id_2, nextmovieid
print('Shape User-Ratings:\t{}'.format(newdf.shape))
newdf.sample(5)

Shape User-Ratings:	(24053764, 4)


,User,Rating,Date,Movie
13085369,851855,2.0,2005-01-20,2470
8067248,2460335,4.0,2004-05-10,1615
8617242,1187280,3.0,2005-10-30,1719
13207828,2120543,3.0,2003-07-12,2499
9021354,546405,2.0,2005-09-22,1798


In [3]:
newdf.head()

,User,Rating,Date,Movie
1,1488844,3.0,2005-09-06,1
2,822109,5.0,2005-05-13,1
3,885013,4.0,2005-10-19,1
4,30878,4.0,2005-12-26,1
5,823519,3.0,2004-05-03,1


In [4]:
ratings=pd.DataFrame(newdf)
ratings.head()

,User,Rating,Date,Movie
1,1488844,3.0,2005-09-06,1
2,822109,5.0,2005-05-13,1
3,885013,4.0,2005-10-19,1
4,30878,4.0,2005-12-26,1
5,823519,3.0,2004-05-03,1


In [5]:
df_title = pd.read_csv('titles.csv', encoding = "ISO-8859-1", header = None, names = ['Movie_Id', 'Year', 'Name'])

In [6]:
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24053764 entries, 1 to 24058262
Data columns (total 4 columns):
User      object
Rating    float64
Date      object
Movie     int64
dtypes: float64(1), int64(1), object(2)
memory usage: 917.6+ MB
None


In [7]:
# Fill NaN values in user_id and movie_id column with 0
ratings['User'] = ratings['User'].fillna(0)
ratings['Movie'] = ratings['Movie'].fillna(0)
# Replace NaN values in rating column with average of all values
ratings['Rating'] = ratings['Rating'].fillna(ratings['Rating'].mean())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24053764 entries, 1 to 24058262
Data columns (total 4 columns):
User      object
Rating    float64
Date      object
Movie     int64
dtypes: float64(1), int64(1), object(2)
memory usage: 917.6+ MB
None


In [8]:
# Randomly sample 80% of the ratings dataset
sampdata = ratings.sample(frac=0.01)
small_data= sampdata[['User','Rating', 'Movie']]
# Check the sample info
small_data.head()

,User,Rating,Movie
13733127,1152249,4.0,2613
19893263,721648,1.0,3801
8381862,980213,4.0,1661
12185694,169980,4.0,2371
18889799,1188347,3.0,3611


In [9]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(small_data, test_size=0.2)

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['User', 'Movie', 'Rating'])
test_data_matrix = test_data.as_matrix(columns = ['User', 'Movie', 'Rating'])
# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix.shape)

(192430, 3)
(48108, 3)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
from sklearn.metrics.pairwise import pairwise_distances
# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

MemoryError: 

In [ ]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:6, :6])

In [ ]:
# Function to predict ratings
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # Use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [ ]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print("Testing")
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))
print("Training")
print('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))